<a href="https://colab.research.google.com/github/yousenwang/langchain_llm/blob/main/Retriever_with_instructor_embed_on_SOP_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install langchain==0.0.215 chromadb pypdf sentence_transformers InstructorEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.5/254.5 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 8.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.1/965.1 kB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [2]:
!pip show langchain

Name: langchain
Version: 0.0.215
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langchainplus-sdk, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [3]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.llms import OpenAI
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
# from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [4]:
loader = DirectoryLoader('./', glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

In [5]:
len(documents)

6

In [6]:
documents[0]

Document(page_content=' \n1 / 6 \nThis document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be \nused to introduce Wareconn functions and procedures. Please do not use it for other purposes.  1 wareconn standard operating procedure  \n \n \n \n \n \n \nwareconn standard operating procedure  \nRMA request  SOP  \n \nVersion : \nVersion  Date  Editor  Description  \nv 1.0  2023/03/16 Eric Sun First draft (English version)  \n \n \nContent  \nSOP ................................ ................................ ................................ ................................  2 \n1.1 RMA request procedure (Parts return)  ................................ ...............  2 \n \n \n \n \n', metadata={'source': 'MSFT RMA request SOP v1.0.pdf', 'page': 0})

# Split Docs into Chunks

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000//2, chunk_overlap=200//2)
texts = text_splitter.split_documents(documents)

In [8]:
len(texts)

12

In [9]:
texts[3]

Document(page_content='1.1 RMA request  procedure (Parts return)  \n1. System login  \n⚫ Module: https://www.wareconn.com/  \n⚫ Description:  Login  with account and password  \n \nTo log in, follow these steps:  \nGo to https://www.wareconn.com/ and click Log in  \n \nFill in your email and password, then click Login\n \nClick Enter  in the Customer portal', metadata={'source': 'MSFT RMA request SOP v1.0.pdf', 'page': 1})

# Embeddings

In [10]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})

load INSTRUCTOR_Transformer
max_seq_length  512


In [11]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [12]:
# persiste the db to disk
vectordb.persist()
vectordb = None


In [13]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

In [14]:
from google.colab import files
files.download(persist_directory)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
print(123)

123


# Retriever

In [16]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})


Q1 :How can I navigate to the RMA request page in the Wareconn Customer Portal? -3

A1: To navigate to the RMA request page, click on "Warranty Claims" in the left menu, then click on the "+Add" button.


In [20]:
q1 = "How can I navigate to the RMA request page in the Wareconn Customer Portal?"
ans1 = retriever.get_relevant_documents(q1)

In [27]:
for i, a in enumerate(ans1):
  print("-"*10+f"Answer {i+1}"+"-"*10)
  print(a.page_content)
  print(a.metadata)

----------Answer 1----------
1.1 RMA request  procedure (Parts return)  
1. System login  
⚫ Module: https://www.wareconn.com/  
⚫ Description:  Login  with account and password  
 
To log in, follow these steps:  
Go to https://www.wareconn.com/ and click Log in  
 
Fill in your email and password, then click Login
 
Click Enter  in the Customer portal
{'source': 'MSFT RMA request SOP v1.0.pdf', 'page': 1}
----------Answer 2----------
3 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  3 wareconn standard operating procedure  
2. Go to  RMA Request  Page 
⚫ Module: Customer portal -Warranty Claims -Warranty Claims  
⚫ Description: Go to RMA request page  
 
To go to the RMA request page, follow these steps:  
Click Warranty Claims  in the left menu, then click +Add
{'source': 'MSFT RMA request SOP v1.0.pdf', 'page': 2}
----------Answe

Q2: What does the Reminder field indicate when submitting an RMA request? -6

Ans2: The Reminder field shows whether the RMA request is approvable (acceptable) for the warranty provider or not.

In [28]:
q2 = "What does the Reminder field indicate when submitting an RMA request?"
ans2 = retriever.get_relevant_documents(q2)

In [29]:
for i, a in enumerate(ans2):
  print("-"*10+f"Answer {i+1}"+"-"*10)
  print(a.page_content)
  print(a.metadata)

----------Answer 1----------
The Reminder field will show you whether this RMA request is approvable  
(acceptable)  for warranty provider  or not.  
Please remember to fill in WAL# in Customer No. field  
Lastly, click Submit  in the top right corner to finish the  RMA requesting process
{'source': 'MSFT RMA request SOP v1.0.pdf', 'page': 5}
----------Answer 2----------
6 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  6 wareconn standard operating procedure  
5. Submit the RMA  request  
 
⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Submit  
⚫ Description: Submit the RMA request  
 
The Reminder field will show you whether this RMA request is approvable
{'source': 'MSFT RMA request SOP v1.0.pdf', 'page': 5}
----------Answer 3----------
4 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Lt

 Q3: What information should be filled in the Customer No. field when submitting an RMA request? -6

 Ans3: When submitting an RMA request, you should fill in the WAL# (Warranty Authorization Number) in the Customer No. field.

In [30]:
q3 = "What information should be filled in the Customer No. field when submitting an RMA request?"
ans3 = retriever.get_relevant_documents(q3)

In [34]:
for i, a in enumerate(ans3):
  print("-"*10+f"Answer {i+1}"+"-"*10)
  print(a.page_content)
  print(a.metadata)

----------Answer 1----------
The Reminder field will show you whether this RMA request is approvable  
(acceptable)  for warranty provider  or not.  
Please remember to fill in WAL# in Customer No. field  
Lastly, click Submit  in the top right corner to finish the  RMA requesting process
{'source': 'MSFT RMA request SOP v1.0.pdf', 'page': 5}
----------Answer 2----------
1.1 RMA request  procedure (Parts return)  
1. System login  
⚫ Module: https://www.wareconn.com/  
⚫ Description:  Login  with account and password  
 
To log in, follow these steps:  
Go to https://www.wareconn.com/ and click Log in  
 
Fill in your email and password, then click Login
 
Click Enter  in the Customer portal
{'source': 'MSFT RMA request SOP v1.0.pdf', 'page': 1}
----------Answer 3----------
4 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  4 wareconn

Q4: What steps should be followed when filling in the request information for an RMA? -5

Ans4: To fill in the request information for an RMA, follow these steps:
- Fill in the Product SN (Serial Number), Product PN (Part Number), and defective reason.
- Select the Rack PN (Part Number) by clicking on "Please select Parts" and choose the desired quantity.
- Click on "Confirm" to proceed.
- Optionally, fill in the Parts SN (Serial Number) for the failure part (not required).
- Click on "Save" to save the information.

In [32]:
q4 = "What steps should be followed when filling in the request information for an RMA?"
ans4 = retriever.get_relevant_documents(q4)

In [35]:
for i, a in enumerate(ans4):
  print("-"*10+f"Answer {i+1}"+"-"*10)
  print(a.page_content)
  print(a.metadata)

----------Answer 1----------
The Reminder field will show you whether this RMA request is approvable  
(acceptable)  for warranty provider  or not.  
Please remember to fill in WAL# in Customer No. field  
Lastly, click Submit  in the top right corner to finish the  RMA requesting process
{'source': 'MSFT RMA request SOP v1.0.pdf', 'page': 5}
----------Answer 2----------
1.1 RMA request  procedure (Parts return)  
1. System login  
⚫ Module: https://www.wareconn.com/  
⚫ Description:  Login  with account and password  
 
To log in, follow these steps:  
Go to https://www.wareconn.com/ and click Log in  
 
Fill in your email and password, then click Login
 
Click Enter  in the Customer portal
{'source': 'MSFT RMA request SOP v1.0.pdf', 'page': 1}
----------Answer 3----------
4 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  4 wareconn

Q5: Can this document be used for purposes other than introducing Wareconn functions and procedures? -3

Ans5: No, this document explicitly states that it should not be used for other purposes. Its sole intention is to introduce Wareconn functions and procedures.

In [36]:
q5 = "Can this document be used for purposes other than introducing Wareconn functions and procedures?"
ans5 = retriever.get_relevant_documents(q5)

In [37]:
for i, a in enumerate(ans5):
  print("-"*10+f"Answer {i+1}"+"-"*10)
  print(a.page_content)
  print(a.metadata)

----------Answer 1----------
1 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  1 wareconn standard operating procedure  
 
 
 
 
 
 
wareconn standard operating procedure  
RMA request  SOP  
 
Version : 
Version  Date  Editor  Description  
v 1.0  2023/03/16 Eric Sun First draft (English version)  
 
 
Content
{'source': 'MSFT RMA request SOP v1.0.pdf', 'page': 0}
----------Answer 2----------
5 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  5 wareconn standard operating procedure  
4. Fill in request info  
⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Fill in info  
⚫ Description:  
Fill in Product SN, Product PN and defective reason  (See the instruction s below 